<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第 2 章：处理文本数据

本笔记本中使用的软件包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.4.0
tiktoken version: 0.7.0


- 本章涵盖数据准备和采样，以便为 LLM 输入数据做好准备

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解词嵌入

- 本节无代码

- 嵌入有多种形式；本书重点介绍文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLM 适用于高维空间（即数千个维度）中的嵌入
- 由于我们无法想象这样的高维空间（我们人类以 1、2 或 3 维思考），下图说明了 2 维嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本分词

- 在本节中，我们将对文本进行标记，即将文本分解为更小的单元，例如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们要处理的原始文本
- [伊迪丝·华顿的《判决》](https://en.wikisource.org/wiki/The_Verdict) 是一部公共领域短篇小说

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

-（如果在执行上一个代码单元时遇到 `ssl.SSLCertVerificationError`，则可能是由于使用了过时的 Python 版本；您可以在 GitHub 上找到[更多信息](https://github.com/rasbt/LLMs-from-scratch/pull/403)

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是为 LLM 标记并嵌入此文本
- 让我们根据一些简单的示例文本开发一个简单的标记器，然后我们可以将其应用于上面的文本
- 以下正则表达式将按空格进行拆分

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅希望根据空格进行拆分，还希望根据逗号和句号进行拆分，因此让我们修改正则表达式来实现这一点

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 正如我们所见，这会创建空字符串，让我们删除它们

In [6]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来不错，但我们还要处理其他类型的标点符号，例如句号、问号等

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这非常好，我们现在准备将此标记应用于原始文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 让我们计算一下 token 的总数

In [9]:
print(len(preprocessed))

4690


## 2.3 将 token 转换为 token ID

- 接下来，我们将文本标记转换为标记 ID，以便稍后通过嵌入层进行处理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从这些 token 中，我们现在可以构建一个包含所有唯一 token 的词汇表

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是该词汇表中的前 50 个条目：

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面，我们来说明如何使用小词汇量对一段简短的示例文本进行标记：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将其全部放入 tokenizer 类中

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 函数将文本转换为 token ID
- `decode` 函数将 token ID 重新转换为文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用标记器将文本编码（即标记化）为整数
- 然后可以（稍后）将这些整数嵌入作为 LLM 的输入

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们可以将整数解码回文本

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文标记

- 为未知单词添加一些“特殊”标记并表示文本的结束很有用

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些标记器使用特殊标记来帮助 LLM 提供额外的上下文
- 其中一些特殊标记是
  - `[BOS]` (beginning of sequence)标记文本的开头
  - `[EOS]` (end of sequence)标记文本的结尾（这通常用于连接多个不相关的文本，例如，两个不同的维基百科文章或两本不同的书，等等）
  - `[PAD]` (padding) 如果我们训练批量大小大于 1 的 LLM（我们可能包含多个不同长度的文本；使用填充标记，我们将较短的文本填充到最长的长度，以便所有文本的长度相等）
- `[UNK]` 表示未包含在词汇表中的单词

- 请注意，GPT-2 不需要上述任何这些标记，而只使用 `<|endoftext|>` 标记来降低复杂性
- `<|endoftext|>` 类似于上面提到的 `[EOS]` 标记
- GPT 也使用 <|endoftext|> 作为填充词元（因为在批量输入训练时我们通常会使用掩码，反正我们不会关注填充的词元，所以这些词元是什么并不重要）
- GPT-2 不使用 <UNK> 词元来处理词汇表之外的单词；相反，GPT-2 使用字节对编码（BPE）分词器，它将单词分解为子词单元，我们将在后面的部分讨论这一点

- 我们在两个独立的文本源之间使用 `<|endoftext|>` 标记：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看如果我们标记以下文本会发生什么：

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上述操作会产生错误，因为词汇表中不包含单词“Hello”
- 为了处理这种情况，我们可以向词汇表添加特殊标记，例如 `"<|unk|>"` 来表示未知单词
- 由于我们已经扩展了词汇表，因此让我们添加另一个名为 `"<|endoftext|>"` 的标记，它在 GPT-2 训练中用于表示文本的结尾（它也用于连接的文本之间，例如如果我们的训练数据集包含多篇文章、书籍等）

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整令牌，以便知道何时以及如何使用新的`<unk>` 标记

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们尝试使用修改后的标记器来标记文本：

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2 使用 BytePair 编码 (BPE) 作为其标记器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元甚至单个字符，从而使其能够处理词汇表之外的单词
- 例如，如果 GPT-2 的词汇表中没有单词“unfamiliarword”，它可能会将其标记为 ["unfam"、"iliar"、"word"] 或其他子词分解，具体取决于其训练过的 BPE 合并
- 原始 BPE 标记器可以在这里找到：[https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用 OpenAI 开源 [tiktoken](https://github.com/openai/tiktoken) 库中的 BPE 标记器，该库在 Rust 中实现其核心算法以改进计算性能
- 我在 [./bytepair_encoder](../02_bonus_bytepair-encoder) 中创建了一个笔记本，对这两种实现进行了并排比较（tiktoken 在示例文本上的速度大约快 5 倍）

In [25]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 标记器将未知单词分解为子单词和单个字符：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 使用滑动窗口进行数据采样

- 我们训练 LLM 每次生成一个单词，因此我们希望相应地准备训练数据，其中序列中的下一个单词代表要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标
- 因为我们希望模型预测下一个单词，所以目标是向右移动一个位置的输入

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 逐一进行预测，结果如下：

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 在介绍完注意力机制后，我们将在后面的章节中处理下一个单词的预测
- 现在，我们实现了一个简单的数据加载器，它遍历输入数据集并返回移位一个的输入和目标

- 安装并导入 PyTorch（安装提示见附录 A）

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.4.0


- 我们使用滑动窗口方法，将位置改变 +1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建数据集和数据加载器，从输入文本数据集中提取块

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们针对上下文大小为 4 的 LLM 测试批量大小为 1 的数据加载器：

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 使用步幅等于上下文长度（此处为 4）的示例，如下所示：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以创建分批输出
- 请注意，我们在这里增加了步幅，这样批次之间就不会出现重叠，因为更多的重叠可能会导致过度拟合

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建 token 嵌入

- 数据已经几乎准备好用于大型语言模型（LLM）
- 但最后让我们使用嵌入层将标记嵌入到连续向量表示中
- 通常，这些嵌入层是 LLM 本身的一部分，并在模型训练期间进行更新（训练）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，输入 ID 分别为 2、3、5 和 1（标记化之后）：

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为了简单起见，假设我们的词汇表只有 6 个单词，并且我们想要创建大小为 3 的嵌入：

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将产生一个 6x3 的权重矩阵：

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码（one-hot encoding）的人来说，上述嵌入层方法本质上只是一种更有效的实现独热编码的方法，然后在全连接层中进行矩阵乘法，这在 [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul) 中的补充代码中进行了描述
- 因为嵌入层只是一种更有效的实现，相当于独热编码和矩阵乘法方法，所以可以将其看作是一个可以通过反向传播进行优化的神经网络层

- 要将 id 为 3 的 token 转换为三维向量，我们执行以下操作：

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上面是 `embedding_layer` 权重矩阵中的第 4 行
- 要嵌入上面所有四个 `input_ids` 值，我们这样做

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一种查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **您可能对比较嵌入层与常规线性层的奖励内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul/README.zh.md)**

## 2.8 编码单词位置

- 嵌入层将 ID 转换为相同的向量表示，无论它们位于输入序列的什么位置：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入与标记嵌入向量相结合，形成大型语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- BytePair 编码器的词汇量为 50,257:
- 假设我们要将输入标记编码为 256 维向量表示：

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中抽样数据，我们会将每批中的标记嵌入到 256 维向量中
- 如果批大小为 8，每批有 4 个标记，则会产生 8 x 4 x 256 张量：

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


- GPT-2 使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


- 要创建 LLM 中使用的输入嵌入，我们只需添加标记和位置嵌入：

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


- 在输入处理工作流程的初始阶段，输入文本被分割成单独的标记
- 分割之后，这些标记将根据预定义的词汇转换为标记 ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

总结和要点

请参阅 [./dataloader.ipynb](./dataloader.zh.ipynb) 代码笔记本，这是我们在本章中实现的数据加载器的简明版本，在接下来的章节中将需要它来训练 GPT 模型。

请参阅 [./exercise-solutions.ipynb](./exercise-solutions.zh.ipynb) 了解练习解决方案。